In [1]:
!pip install tensorflow==2.14

In [2]:
!pip install -q -U google-generativeai

In [3]:
!pip install -q google-cloud-aiplatform==1.36.0 google-cloud-discoveryengine==0.11.2 langchain==0.0.327 p

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.47 requires langchain<0.1,>=0.0.350, but you have langchain 0.0.327 which is incompatible.
trulens-eval 0.19.2 requires langchain>=0.0.335, but you have langchain 0.0.327 which is incompatible.


In [ ]:
!pip -q install langchain_experimental langchain_core
!pip -q install google-generativeai==0.3.1
!pip -q install google-ai-generativelanguage==0.4.0
!pip -q install langchain-google-genai
!pip -q install trulens_eval
!pip -q install google

In [ ]:
import os
if not os.path.exists('new_articles'):
    os.mkdir('new_articles')

In [ ]:
# !wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
# !unzip -q new_articles.zip -d new_articles

In [ ]:
import pathlib
import textwrap
import os
import google.generativeai as genai
import pathlib
import textwrap
from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
load_dotenv()
GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
%%time
response=model.generate_content("What is the meaning of life?")

In [ ]:
to_markdown(response.text)
print(response.text)

### RAG(Lanchain+chroma+GeminiAI)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate

# from langchain.schema.output_parser import StrOutputParser

In [ ]:
loader = DirectoryLoader('./new_articles/', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embeddings)

### RAG template class for True Lense

In [ ]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()

In [ ]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        retriever = vectordb.as_retriever()
        results = retriever.get_relevant_documents(query) 
        return results[0]  

    
    @instrument
    def generate_completion(self, query:str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        message = [{
            "role": "user",
            "parts": [
                f"We have provided context information below. \n"
                f"---------------------\n"
                f"{context_str}"
                f"\n---------------------\n"
                f"Given this information, please answer the question: {query}"
            ]
        }]
        model = genai.GenerativeModel('gemini-pro')
        response = model.generate_content(message)
        response_txt = response.text
        return response_txt
        

    @instrument
    def query(self, query:str ) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion
    
rag = RAG_from_scratch()


In [ ]:
# # Usage
# rag_instance = RAG_from_scratch()
# my_query = "What is the news about Pando?"
# result = rag_instance.query(my_query)
# print(result)



In [ ]:
#Evaluation using TrueLens


In [ ]:
# !pip install trulens-eval

In [ ]:
!pip install streamlit_javascript


In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-"

In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np
import streamlit as st


In [ ]:
# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

In [ ]:
with tru_rag as recording:
    rag.query("What is the news about Pando")

In [ ]:
tru.get_leaderboard(app_ids=["RAG v1"])

In [ ]:
tru.run_dashboard()